In [1]:
# =========================================================
# FAKE NEWS DETECTION – MiniLM + PROMPT TUNING x FAKENEWSNET
# (Đã cấu hình chuẩn cho nreimers/MiniLM-L6-H384-uncased)
# =========================================================

# 1. CÀI ĐẶT THƯ VIỆN
!pip install -q transformers datasets peft accelerate bitsandbytes scikit-learn pandas numpy psutil

import os, re, shutil, psutil, warnings
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, EarlyStoppingCallback,
    DataCollatorWithPadding
)
from peft import PromptTuningConfig, PromptTuningInit, get_peft_model, TaskType
from google.colab import drive

warnings.filterwarnings("ignore")

# 2. KIỂM TRA GPU & MOUNT DRIVE
if torch.cuda.is_available():
    print(f"✅ Device: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ CẢNH BÁO: Đang chạy CPU! Quá trình train sẽ rất chậm.")

drive.mount('/content/drive', force_remount=True)
OUTPUT_DIR = "/content/drive/MyDrive/FakeNewsNet_MiniLM_PromptTuning"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 3. TẢI DATASET
print("\n⏳ Đang tải dataset FakeNewsNet...")
try:
    ds_gossip = load_dataset("rickstello/FakeNewsNet", "gossipcop", split="train")
    ds_politi = load_dataset("rickstello/FakeNewsNet", "politifact", split="train")
    dataset_full = concatenate_datasets([ds_gossip, ds_politi])
    df = pd.DataFrame(dataset_full)
except Exception as e:
    print(f"⚠️ Tải config con thất bại ({e}), tải bản default...")
    dataset = load_dataset("rickstello/FakeNewsNet", split="train")
    df = pd.DataFrame(dataset)

# 4. XỬ LÝ DỮ LIỆU AN TOÀN
# Tìm tên cột tự động
text_col = next((c for c in ['news_content', 'text', 'content', 'body'] if c in df.columns), None)
title_col = next((c for c in ['title', 'news_title', 'headline'] if c in df.columns), None)
label_col = next((c for c in ['real', 'label', 'class', 'fake'] if c in df.columns), None)

print(f"🔍 Detected Columns: Text='{text_col}' | Title='{title_col}' | Label='{label_col}'")

if label_col is None:
    raise ValueError("❌ Dataset không có cột nhãn!")

df['label'] = df[label_col]

# Xử lý nội dung: Ưu tiên Title + Text
title_data = df[title_col].astype(str).fillna('') if title_col else pd.Series([''] * len(df))
text_data = df[text_col].astype(str).fillna('') if text_col else pd.Series([''] * len(df))

if text_col is None:
    print("⚠️ CẢNH BÁO: Không tìm thấy cột nội dung. Chỉ học từ Tiêu đề.")

def clean_text(s):
    if not isinstance(s, str): return ""
    s = s.lower()
    s = re.sub(r'https?://\S+', ' ', s)
    s = re.sub(r'<.*?>', ' ', s)
    s = re.sub(r'[^a-z0-9\s]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

print("🧹 Đang làm sạch dữ liệu...", end="")
df['content'] = (title_data + " [SEP] " + text_data).apply(clean_text)
df = df[df['content'].str.len() > 10].drop_duplicates(subset=['content'])
print(f" → Hoàn tất. Số lượng mẫu: {len(df):,}")

# Class Weights
classes = np.unique(df['label'])
class_weights = compute_class_weight('balanced', classes=classes, y=df['label'])
class_weight_dict = {k: float(v) for k, v in zip(classes, class_weights)}
print("Class weights:", class_weight_dict)

# Split Data
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

dataset_dict = DatasetDict({
    "train": Dataset.from_pandas(train_df[['content','label']].reset_index(drop=True)),
    "validation": Dataset.from_pandas(val_df[['content','label']].reset_index(drop=True)),
    "test": Dataset.from_pandas(test_df[['content','label']].reset_index(drop=True))
})

# 5. TOKENIZER (MiniLM)
MODEL_NAME = "nreimers/MiniLM-L6-H384-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(batch["content"], truncation=True, max_length=384, padding=False)

tokenized = dataset_dict.map(tokenize_fn, batched=True, remove_columns=['content'])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 6. MODEL & PROMPT TUNING (Mapping chuẩn cho BERT/MiniLM)
print(f"\n⚙️ Đang khởi tạo {MODEL_NAME} với Prompt Tuning...")
base_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
base_model.config.id2label = {0: "Fake", 1: "Real"}
base_model.config.label2id = {"Fake": 0, "Real": 1}

# --- CẤU HÌNH PROMPT TUNING CHO MINILM ---
peft_config = PromptTuningConfig(
    task_type=TaskType.SEQ_CLS,
    prompt_tuning_init=PromptTuningInit.TEXT,
    prompt_tuning_init_text="Classify if this news article is real or fake:",
    num_virtual_tokens=32,
    tokenizer_name_or_path=MODEL_NAME,

    # === THAM SỐ QUAN TRỌNG CHO MINILM (Kiến trúc BERT) ===
    # MiniLM dùng tên biến khác DistilBERT
    num_layers=base_model.config.num_hidden_layers,      # Thay vì n_layers
    token_dim=base_model.config.hidden_size,             # Thay vì dim
    num_attention_heads=base_model.config.num_attention_heads # Thay vì n_heads
)

model = get_peft_model(base_model, peft_config)
model.print_trainable_parameters()

# 7. CUSTOM TRAINER
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        w = torch.tensor(list(class_weight_dict.values()), dtype=torch.float32, device=model.device)
        loss_fct = torch.nn.CrossEntropyLoss(weight=w)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# 8. TRAIN
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=1,
    learning_rate=1e-2, # LR cao cho Prompt Tuning
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=torch.cuda.is_available(),
    logging_steps=50,
    report_to="none"
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
)

print("\n🚀 BẮT ĐẦU HUẤN LUYỆN (MiniLM + Prompt Tuning)...")
trainer.train()

# 9. ĐÁNH GIÁ
print("\n🎯 KẾT QUẢ TRÊN TEST SET:")
results = trainer.evaluate(tokenized["test"])
print(results)

final_path = os.path.join(OUTPUT_DIR, "final_minilm_pt_model")
trainer.save_model(final_path)
print(f"\n✅ Đã lưu model tại: {final_path}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.5 MB/s eta 0:00:00
✅ Device: Tesla T4
Mounted at /content/drive

⏳ Đang tải dataset FakeNewsNet...


README.md:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

⚠️ Tải config con thất bại (BuilderConfig 'gossipcop' not found. Available: ['default']), tải bản default...


FakeNewsNet.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/23196 [00:00<?, ? examples/s]

...

trainable params: 13,058 || all params: 22,727,044 || trainable%: 0.0575

🚀 BẮT ĐẦU HUẤN LUYỆN (MiniLM + Prompt Tuning)...


model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.631700,0.586973,0.709351,0.726858,0.768718,0.709351
2,0.611500,0.587010,0.766467,0.772573,0.781552,0.766467
3,0.605900,0.566406,0.730539,0.746157,0.783853,0.730539
4,0.594800,0.566989,0.754491,0.766391,0.791475,0.754491
5,0.559400,0.557231,0.728696,0.744920,0.785774,0.728696



🎯 KẾT QUẢ TRÊN TEST SET:


{'eval_loss': 0.6137063503265381, 'eval_accuracy': 0.7392906494702902, 'eval_f1': 0.7485124289071018, 'eval_precision': 0.7632545278134222, 'eval_recall': 0.7392906494702902, 'eval_runtime': 0.9916, 'eval_samples_per_second': 2189.454, 'eval_steps_per_second': 68.578, 'epoch': 5.0}

✅ Đã lưu model tại: /content/drive/MyDrive/FakeNewsNet_MiniLM_PromptTuning/final_minilm_pt_model
